In [87]:
from torch import nn, Tensor
from typing import Any, Callable, Optional, TypeVar, Dict, Union, Tuple
from torch.nn.parameter import Parameter
from torch.utils.hooks import RemovableHandle


### nn.Module의 Method
- \_\_init\_\_
- \_forward_unimplemented
- register_buffer
- register_parameter
- add_module
- get_submodule
- get_parameter
- get_buffer
- apply
- register_forward_pre_hook
- register_forward_hook
- register_full_backward_hook

- \_\_call\_\_, _call_impl

- zero_grad
- extra_repr
- \_\_repr\_\_

- parameters
- named_parameters
- buffers
- named_buffers
- children
- named_children
- modules
- named_modules

이 외에도 많은 Method들이 있다.

### nn.Module 공식 문서의 설명
- Base class for all neural network modules.   
    \> 모든 뉴럴네트워크의 모듈들은 nn.Module 을 base class 로 가진다.
      
      
- Your models should also subclass this class.   
    \> 사용자가 만드는 모델들 또한 이 class를 subclass로 가진다


- Modules can also contain other Modules, allowing to nest them in a tree structure  
    \> Modules는 다른 Modules를 포함할 수 있다.



### 1. \_\_init\_\_( )
- torch.\_C.\_log\_api\_usage\_once : 이 부분은 log를 남기는 것으로 Facebook으로 가는 것으로 [추정된다.](https://discuss.pytorch.org/t/what-does-torch-c-log-api-usage-once-do/137732)


- 다른 class의 \_\_init\_\_ 의 역할 처럼 생성자의 역할을 하여준다.


- training의 default 값은 True이다.


- parameters, buffers, backward_hooks, forward_hooks, forward_pre_hooks등은 모두 비어있는 OrderDict()를 할당해준다.

In [42]:
def __init__(self) -> None:
    """
    Initializes internal Module state, shared by both nn.Module and ScriptModule.
    """
    torch._C._log_api_usage_once("python.nn_module")

    self.training = True
    self._parameters: Dict[str, Optional[Parameter]] = OrderedDict()
    self._buffers: Dict[str, Optional[Tensor]] = OrderedDict()
    self._non_persistent_buffers_set: Set[str] = set()
    self._backward_hooks: Dict[int, Callable] = OrderedDict()
    self._is_full_backward_hook = None
    self._forward_hooks: Dict[int, Callable] = OrderedDict()
    self._forward_pre_hooks: Dict[int, Callable] = OrderedDict()
    self._state_dict_hooks: Dict[int, Callable] = OrderedDict()
    self._load_state_dict_pre_hooks: Dict[int, Callable] = OrderedDict()
    self._modules: Dict[str, Optional['Module']] = OrderedDict()

> <code>self._forward_hooks: Dict[int, Callable] = OrderedDict()</code> 같은 구조는 typing 모듈로 타입 표시하는 방법 중 하나이다. [참고](https://www.daleseo.com/python-typing/)  
즉 forward_hooks는 int를 key로 Callable한 value를 가져야만한다.

### 2. forward( )

In [43]:
forward: Callable[..., Any] = _forward_unimplemented

def _forward_unimplemented(self, *input: Any) -> None:
    r"""Defines the computation performed at every call.

    Should be overridden by all subclasses.

    .. note::
        Although the recipe for forward pass needs to be defined within
        this function, one should call the :class:`Module` instance afterwards
        instead of this since the former takes care of running the
        registered hooks while the latter silently ignores them.
    """
    raise NotImplementedError


\_\_init\_\_과 더불어 PyTorch에서 반드시 forward는 모델의 계산을 정의한다. (backward 계산은 backward()를 이용하여 알아서 수행해준다)  
forward을 override 하지 않을 시 \_forward_unimplemented method가 NotImplementedError 에러를 일으킨다

### 3. register_buffer( ), register_parameter( )

In [59]:
def register_buffer(self, name: str, tensor: Optional[Tensor], persistent: bool = True) -> None:      
        if persistent is False and isinstance(self, torch.jit.ScriptModule):
            raise RuntimeError("ScriptModule does not support non-persistent buffers")

        if '_buffers' not in self.__dict__:
            raise AttributeError(
                "cannot assign buffer before Module.__init__() call")
        elif not isinstance(name, torch._six.string_classes):
            raise TypeError("buffer name should be a string. "
                            "Got {}".format(torch.typename(name)))
        elif '.' in name:
            raise KeyError("buffer name can't contain \".\"")
        elif name == '':
            raise KeyError("buffer name can't be empty string \"\"")
        elif hasattr(self, name) and name not in self._buffers:
            raise KeyError("attribute '{}' already exists".format(name))
        elif tensor is not None and not isinstance(tensor, torch.Tensor):
            raise TypeError("cannot assign '{}' object to buffer '{}' "
                            "(torch Tensor or None required)"
                            .format(torch.typename(tensor), name))
        else:
            self._buffers[name] = tensor
            if persistent:
                self._non_persistent_buffers_set.discard(name)
            else:
                self._non_persistent_buffers_set.add(name)

- 모듈에 buffer를 추가한다.
- 일반적으로 model parameter로 고려 되어서는 안되는 buffer를 등록하는데 사용한다.
- 예를들어 BatchNorm의 'running_mean'은 parameter는 아니지만 Module의 일부이다.
- persistent를는 buffer를 Model의 일부로 영구적 or 비영구적으로 가져갈 것인지 결정한다.

In [62]:
def register_parameter(self, name: str, param: Optional[Parameter]) -> None:        
        if '_parameters' not in self.__dict__:
            raise AttributeError(
                "cannot assign parameter before Module.__init__() call")

        elif not isinstance(name, torch._six.string_classes):
            raise TypeError("parameter name should be a string. "
                            "Got {}".format(torch.typename(name)))
        elif '.' in name:
            raise KeyError("parameter name can't contain \".\"")
        elif name == '':
            raise KeyError("parameter name can't be empty string \"\"")
        elif hasattr(self, name) and name not in self._parameters:
            raise KeyError("attribute '{}' already exists".format(name))

        if param is None:
            self._parameters[name] = None
        elif not isinstance(param, Parameter):
            raise TypeError("cannot assign '{}' object to parameter '{}' "
                            "(torch.nn.Parameter or None required)"
                            .format(torch.typename(param), name))
        elif param.grad_fn:
            raise ValueError(
                "Cannot assign non-leaf Tensor to parameter '{0}'. Model "
                "parameters must be created explicitly. To express '{0}' "
                "as a function of another Tensor, compute the value in "
                "the forward() method.".format(name))
        else:
            self._parameters[name] = param

- 모듈에 parameter를 추가한다.
- 주어진 name을 사용해 attribute로 access 할 수 있다.
- name : parameter의 이름
- param : 모듈에 추가할 parameter

### 4. add_module( )

In [63]:
def add_module(self, name: str, module: Optional['Module']) -> None:
    if not isinstance(module, Module) and module is not None:
        raise TypeError("{} is not a Module subclass".format(
            torch.typename(module)))
    elif not isinstance(name, torch._six.string_classes):
        raise TypeError("module name should be a string. Got {}".format(
            torch.typename(name)))
    elif hasattr(self, name) and name not in self._modules:
        raise KeyError("attribute '{}' already exists".format(name))
    elif '.' in name:
        raise KeyError("module name can't contain \".\", got: {}".format(name))
    elif name == '':
        raise KeyError("module name can't be empty string \"\"")
    self._modules[name] = module

- 현재 Module에 새로운 Module을 추가한다.

### 5. get_submodule( ), get_parameter( ), get_buffer( )

In [64]:
  def get_submodule(self, target: str) -> "Module":
        if target == "":
            return self

        atoms: List[str] = target.split(".")
        mod: torch.nn.Module = self

        for item in atoms:

            if not hasattr(mod, item):
                raise AttributeError(mod._get_name() + " has no "
                                     "attribute `" + item + "`")

            mod = getattr(mod, item)

            if not isinstance(mod, torch.nn.Module):
                raise AttributeError("`" + item + "` is not "
                                     "an nn.Module")

        return mod

- target 인자로 주어진 submodule을 반환한다.
- 없다면 AttributeError를 발생시킨다.

In [66]:
def get_parameter(self, target: str) -> "Parameter":
        module_path, _, param_name = target.rpartition(".")

        mod: torch.nn.Module = self.get_submodule(module_path)

        if not hasattr(mod, param_name):
            raise AttributeError(mod._get_name() + " has no attribute `"
                                 + param_name + "`")

        param: torch.nn.Parameter = getattr(mod, param_name)

        if not isinstance(param, torch.nn.Parameter):
            raise AttributeError("`" + param_name + "` is not an "
                                 "nn.Parameter")

        return param

In [67]:
def get_buffer(self, target: str) -> "Tensor":        
        module_path, _, buffer_name = target.rpartition(".")

        mod: torch.nn.Module = self.get_submodule(module_path)

        if not hasattr(mod, buffer_name):
            raise AttributeError(mod._get_name() + " has no attribute `"
                                 + buffer_name + "`")

        buffer: torch.Tensor = getattr(mod, buffer_name)

        if buffer_name not in mod._buffers:
            raise AttributeError("`" + buffer_name + "` is not a buffer")

        return buffer

- target 인자로 주어진 buffer를 반환한다
- 경로가 잘못되거나 buffer아 아니면 AttributeError를 일으킨다.

### 6. apply

In [73]:
T = TypeVar('T', bound='Module')

def apply(self: T, fn: Callable[['Module'], None]) -> T:
    for module in self.children():
        module.apply(fn)
    fn(self)
    return self

- 현재 module의 모든 childern에 fn(추가하고싶은 함수)을 추가한다. 
- model parameter를 초기화할 때 자주 쓴다.

### 7. register_backward_hook( ), register_forward_pre_hook( ), register_forward_hook( )


In [88]:
_global_backward_hooks: Dict[int, Callable] = OrderedDict()
_global_is_full_backward_hook: Optional[bool] = None
_global_forward_pre_hooks: Dict[int, Callable] = OrderedDict()
_global_forward_hooks: Dict[int, Callable] = OrderedDict()
    
_grad_t = Union[Tuple[Tensor, ...], Tensor]    

추후에 모든 모듈에 공통되는 hook를 추적하기 위해 전역 상태로 만든다(?)

In [81]:
def register_forward_pre_hook(self, hook: Callable[..., None]) -> RemovableHandle:
        handle = hooks.RemovableHandle(self._forward_pre_hooks)
        self._forward_pre_hooks[handle.id] = hook
        return handle

In [82]:
def register_forward_hook(self, hook: Callable[..., None]) -> RemovableHandle:
        handle = hooks.RemovableHandle(self._forward_hooks)
        self._forward_hooks[handle.id] = hook
        return handle

#### register_forward_pre_hook 
- self.\_forward\_pre\_hooks에 hook을 저장한다.
- forward_pre_hook은 __forward가 호출 되기전 hook이 호출__ 된다
- input을 수정이 할 수 있다.
- 반환 할때는 tuple로 warp 한다.

####  register_forward_hook
- self.\_forward\_hooks에 hook을 저장한다.
- forward_hook은 __forward가 계산을 끝낸 후 hook이 호출__ 된다.
- output을 수정 가능하다.(input도 수정이 가능하다 이미 output이 나와있으므로 forward에 영향을 미치지않는다)

In [90]:
 def register_full_backward_hook(
        self, hook: Callable[['Module', _grad_t, _grad_t], Union[None, Tensor]]
    ) -> RemovableHandle:      
        if self._is_full_backward_hook is False:
            raise RuntimeError("Cannot use both regular backward hooks and full backward hooks on a "
                               "single Module. Please use only one of them.")

        self._is_full_backward_hook = True

        handle = hooks.RemovableHandle(self._backward_hooks)
        self._backward_hooks[handle.id] = hook
        return handle

####  register_full_backward_hook
- 이미 self.\_is_full_backward_hook 상태이면 Error를 반환하고 아니면 True로 변경한다
- self_backward_hooks에 hook을 저장한다.
- Module에 모든 input에 대한 gradient 생성시마다 hook가 호출된다
- grad_input과 grad_output은 gradient를 포함하는 튜플이다.
- hook는 argument를 수정해서는 안 되지만 grad_input 대신에 사용될 입력과 관련하여 선택적으로 새 gradient를 반환할 수 있다.
- RemovableHandle는 추가된 hook를 삭제하는 기능을 해준다.

### 8. \_\_call\_\_ ,\_call_impl

In [100]:
def _call_impl(self, *input, **kwargs):
        forward_call = (self._slow_forward if torch._C._get_tracing_state() else self.forward)
        # hook가 없다면 이부분은 건너 띄고 forward만호출한다
        # If we don't have any hooks, we want to skip the rest of the logic in
        # this function, and just call forward.
        if not (self._backward_hooks or self._forward_hooks or self._forward_pre_hooks or _global_backward_hooks
                or _global_forward_hooks or _global_forward_pre_hooks):
            return forward_call(*input, **kwargs)
        # Do not call functions when jit is used
        full_backward_hooks, non_full_backward_hooks = [], []
        if self._backward_hooks or _global_backward_hooks:
            full_backward_hooks, non_full_backward_hooks = self._get_backward_hooks()
        if _global_forward_pre_hooks or self._forward_pre_hooks:
            for hook in (*_global_forward_pre_hooks.values(), *self._forward_pre_hooks.values()):
                result = hook(self, input)
                if result is not None:
                    if not isinstance(result, tuple):
                        result = (result,)
                    input = result

        bw_hook = None
        if full_backward_hooks:
            bw_hook = hooks.BackwardHook(self, full_backward_hooks)
            input = bw_hook.setup_input_hook(input)

        result = forward_call(*input, **kwargs)
        if _global_forward_hooks or self._forward_hooks:
            for hook in (*_global_forward_hooks.values(), *self._forward_hooks.values()):
                hook_result = hook(self, input, result)
                if hook_result is not None:
                    result = hook_result

        if bw_hook:
            result = bw_hook.setup_output_hook(result)

        # Handle the non-full backward hooks
        if non_full_backward_hooks:
            var = result
            while not isinstance(var, torch.Tensor):
                if isinstance(var, dict):
                    var = next((v for v in var.values() if isinstance(v, torch.Tensor)))
                else:
                    var = var[0]
            grad_fn = var.grad_fn
            if grad_fn is not None:
                for hook in non_full_backward_hooks:
                    wrapper = functools.partial(hook, self)
                    functools.update_wrapper(wrapper, hook)
                    grad_fn.register_hook(wrapper)
                self._maybe_warn_non_full_backward_hook(input, result, grad_fn)

        return result

    __call__ : Callable[..., Any] = _call_impl

        

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 53)

- \_\_call\_\_은 인스턴스가 호출될때 사용되는 Python의 magic method이다.
- PyTorch에서는 \_call_impl로 오버라이딩(?) 하였다.
- <code> if not (self._backward_hooks or self._forward_hooks or self._forward_pre_hooks or _global_backward_hooks
or _global_forward_hooks or _global_forward_pre_hooks)...</code> 에서  보다시피 hook이 없다면 건너 띄고 forward를 수행하게 된다

### 9. train( )

In [102]:
def train(self: T, mode: bool = True) -> T:
    r"""Sets the module in training mode.

    This has any effect only on certain modules. See documentations of
    particular modules for details of their behaviors in training/evaluation
    mode, if they are affected, e.g. :class:`Dropout`, :class:`BatchNorm`,
    etc.

    Args:
        mode (bool): whether to set training mode (``True``) or evaluation
                     mode (``False``). Default: ``True``.

    Returns:
        Module: self
    """
    if not isinstance(mode, bool):
        raise ValueError("training mode is expected to be boolean")
    self.training = mode
    for module in self.children():
        module.train(mode)
    return self

- True일시, Module을 training mode로 설정한다
- init에서 True로 설정한 self.training값을 변경한다
- 모든 Module에 영향을 미치는 것은 아니라고 한다.(영향을 받는 경우 : Dropout, BatchNorm등)

### 10. zero_grad( )

In [104]:
def zero_grad(self, set_to_none: bool = False) -> None:
        if getattr(self, '_is_replica', False):
            warnings.warn(
                "Calling .zero_grad() from a module created with nn.DataParallel() has no effect. "
                "The parameters are copied (in a differentiable manner) from the original module. "
                "This means they are not leaf nodes in autograd and so don't accumulate gradients. "
                "If you need gradients in your forward method, consider using autograd.grad instead.")

        for p in self.parameters():
            if p.grad is not None:
                if set_to_none:
                    p.grad = None
                else:
                    if p.grad.grad_fn is not None:
                        p.grad.detach_()
                    else:
                        p.grad.requires_grad_(False)
                    p.grad.zero_()

- 모든 Model의 parameters를 0으로 한다
- torch.optim.Optimizer와 유사하다.
- set_to_none 는 grads를 None으로 변경할 수 있는 parameter이다.(torch.optim.Optimizer.zero_grad에서 더 자세히 확인가능)

### 11. extra_repr( ), \_\_repr\_\_

In [109]:
def extra_repr(self) -> str:
    
    return ''

- extra_repr은 PyTorch에서 repr을 표현하는 데 사용하는 method이다.
- 빈문자열인 이유는 각 사용자가 자신이 만든 Module 에 맞는 설명을 apply 함수로 채워 넣으라는 의미인거 같다.
- 그리고 직접적으로 출력을 해주는 것은 아래 \_\_repr\_\_ method 를 거쳐야한다.

In [107]:
def __repr__(self):
        # We treat the extra repr like the sub-module, one item per line
        extra_lines = []
        extra_repr = self.extra_repr()
        # empty string will be split into list ['']
        if extra_repr:
            extra_lines = extra_repr.split('\n')
        child_lines = []
        for key, module in self._modules.items():
            mod_str = repr(module)
            mod_str = _addindent(mod_str, 2)
            child_lines.append('(' + key + '): ' + mod_str)
        lines = extra_lines + child_lines

        main_str = self._get_name() + '('
        if lines:
            # simple one-liner info, which most builtin Modules will use
            if len(extra_lines) == 1 and not child_lines:
                main_str += extra_lines[0]
            else:
                main_str += '\n  ' + '\n  '.join(lines) + '\n'

        main_str += ')'
        return main_str

- \_\_repr\_\_ 은 Python의 Magic Method 중 하나이다. str과도 비슷하지만 str은 있는 그대로의 문자열을 반환해주는 역할이라면 repr은 좀 더 사용자에의 이해를 돕는 방향으로 객체를 표현해준다.
- \_\_repr\_\_의 역할은 간단하다. self.extra_repr( )을 가져와서 출력해주는 역할을 한다.

### 12. 그 외 메서드

- parameters
- named_parameters
- buffers
- named_buffers
- children
- named_children
- modules
- named_modules

각각 해당 모듈의 parameters, buffers, childern, modules를 보여준다는 특징이 있다.  
앞에 named_ 가 붙으면 iteration에서 name, module로 분리되어 나오기 때문에 원하는 name에 해당하는 module을 골라낼 수 있다.